#  Data Fusion All

En este notebook se procederá a la fusion del dataset generado en el notebook 2 (hits) con el generado en el 3 (canciones aleatorias)

Para ello se concatenaran ambos dataframe y se eliminaran todas las duplicidades.
1. En caso de que dataframe de hits tenga duplicidades, se ordenará por semanas en lista, y se eliminaran las duplicidades que menos tiempo tienen.
2. En caso de que dataframe de canciones aleatorias tenga duplicidades, nos quedaremos con la primera encontrada.
3. En caso de que haya una duplicidad de un hit con una cancion aleatoria, se descartará la aleatoria.

Ya que tenemos un mayor numero de canciones aleatorias, una vez seleccionadas aquellas que no son hits. seleccionaremos la misma cantidad de canciones aleatoriamente que hits tenemos, para asi tener un dataset balanceado.


In [54]:
import pandas as pd
import numpy as np

In [55]:
df_random = pd.read_csv("../Data/random_songs_uniq_1958_2020.csv")

In [56]:
df_random.shape

(69067, 23)

In [57]:
df_random.head()

,artist,title,id,release_date,collaboration,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms,popularity_artist,popularity_song,genres,album,label
0,Airbourne,It Ain't Over Till It's Over,68SzcK5s3H4RXEyyJtdFAO,2010,False,0.484,0.999,9,-2.336,1,0.1840,0.000271,0.108000,0.1590,0.0323,111.019,4,197706,59,33,"['glam metal', 'hard rock', 'metal', 'rock', '...",No Guts. No Glory,Roadrunner Records
1,Slow Meadow,Hurricane,55V1T1cUIWJYJPI77asAat,2017-11-17,False,0.117,0.264,2,-13.967,1,0.0377,0.833000,0.936000,0.0762,0.0696,79.146,4,331008,54,0,"['calming instrumental', 'chamber pop', 'compo...",Costero,Hammock Music
2,Green Day,¿Viva La Gloria? (Little Girl),2mnk1GHN0XmznKiucY4ZW4,2009-05-15,False,0.519,0.845,5,-4.415,0,0.0281,0.025200,0.000002,0.0922,0.5320,94.955,4,227680,82,55,"['permanent wave', 'pop punk', 'punk']",21st Century Breakdown,Reprise
3,Bart Wolffe,Three Questions by Leo Tolstoy,2gjKSPZulrHzqHQXmKksq9,2014-02-24,False,0.666,0.254,1,-15.736,1,0.9480,0.339000,0.000000,0.1160,0.5090,75.985,4,654342,3,4,[],10 Minute Short Stories,Saland Publishing
4,Satin Jackets,Little Airplanes,5lzq0LoVd0y5YPLlc64FMz,2020-05-29,False,0.546,0.655,0,-7.540,1,0.0286,0.014500,0.752000,0.1090,0.2090,117.001,4,330614,61,48,"['alternative dance', 'deep house', 'indie sou...",Little Airplanes,Eskimo Recordings


In [58]:
df_hits = pd.read_csv ("../Data/top_100_Billboard_1958_2020.csv",dtype={'song_lyrics':'string'})

Se ordena por mayor numero de semanas en "hot-100" y se eliminan duplicados quedandonos con el primero

In [59]:
df_hits_uniq = df_hits.sort_values("weeks", ascending=False).drop_duplicates(subset = ["id"])
# las busquedas en Spotipy, a traves de artista y titulo, pueden encontrar la misma cancion con Ids diferentes,
# ya que la canción puede encontrarse en un album y como single y por tanto los ids no coincidiran aunque es la misma canción
# por ello nos aseguramos eliminando los duplicados que tengan mismo artista y titulo de la canción, para solventar esta
# casuistica.
df_hits_uniq = df_hits_uniq.drop_duplicates(subset = ["artist","title"], keep="first")

In [60]:
len(df_hits_uniq)

25567

In [61]:
df_concact = pd.concat([df_hits_uniq, df_random])
df_concact = df_concact.drop_duplicates(subset = "id", keep="first")
# las busquedas en Spotipy, a traves de artista y titulo, pueden encontrar la misma cancion con Ids diferentes,
# ya que la canción puede encontrarse en un album y como single y por tanto los ids no coincidiran aunque es la misma canción
# por ello nos aseguramos eliminando los duplicados que tengan mismo artista y titulo de la canción, para solventar esta
# casuistica.
df_concact = df_concact.drop_duplicates(subset = ["artist","title"], keep="first")

In [62]:
pd.options.display.max_columns = None

In [63]:
df_concact.head()

,artist,title,id,year_chart,date_chart,release_date,collaboration,rank,weeks,isNew,peakPos,lastPos,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms,popularity_artist,popularity_song,genres,album,label,song_lyrics,popularity
268122,Imagine Dragons,Radioactive,4G8gkOterJn0Ywt6uhqbhp,2014.0,2014-05-10,2012-09-04,False,49.0,87.0,False,3.0,48.0,0.448,0.784,9,-3.686,1,0.0627,0.1060,0.000108,0.6680,0.236,136.245,4,186813,90,72,"['modern rock', 'pop', 'rock']",Night Visions,Kid Ina Korner / Interscope,Whoah-oh\nWhoah-oh\nWhoah-oh\nWhoah\n\nI'm wak...,NaN
267426,AWOLNATION,Sail,5xhQChGGhKLWqBqX4XhtYE,2014.0,2014-03-22,2011,False,45.0,79.0,False,17.0,41.0,0.825,0.435,1,-9.582,1,0.0568,0.4520,0.609000,0.0953,0.243,119.038,4,259102,70,75,"['la indie', 'modern alternative rock', 'moder...",Megalithic Symphony Deluxe,Red Bull Records,Sail!\n\nThis is how I show my love\nI made it...,NaN
244363,Jason Mraz,I'm Yours,1EzrEOXmMH3G43AXT1y7pA,2009.0,2009-10-10,2008-05-12,False,48.0,76.0,False,6.0,49.0,0.686,0.457,11,-8.322,1,0.0468,0.5950,0.000000,0.1050,0.718,150.953,4,242186,81,82,"['acoustic pop', 'neo mellow', 'pop', 'pop rock']",We Sing. We Dance. We Steal Things.,Atlantic Records/ATG,"Well, you done done me, and you bet I felt it\...",NaN
188602,LeAnn Rimes,How Do I Live,7BD50ATrF3Vab5FQy7vtK8,1998.0,1998-10-10,2003-11-18,False,45.0,69.0,False,2.0,48.0,0.577,0.462,2,-7.989,1,0.0279,0.1280,0.000000,0.0822,0.258,128.303,4,266973,64,68,"['country', 'country dawn', 'dance pop', 'pop ...",Greatest Hits,Curb Records,How do I get through one night without you?\nI...,NaN
258774,LMFAO,Party Rock Anthem,0IkKz2J93C94Ei4BvDop7P,2012.0,2012-07-21,2011-01-01,True,49.0,68.0,False,1.0,50.0,0.750,0.727,5,-4.210,0,0.1420,0.0189,0.000000,0.2660,0.359,129.993,4,262173,72,68,"['dance pop', 'edm', 'pop', 'pop rap']",Sorry For Party Rocking,Will I Am / A&M,Party Rock\nYeah\nWoo!\nLet's go!\n\nParty roc...,NaN


In [64]:
df_filter = df_concact.iloc[:2 * len(df_hits_uniq)]

In [65]:
df_target = df_filter.assign(target = np.where(pd.isnull(df_filter['rank']), 0, 1))

In [66]:
df_target.shape

(51134, 33)

In [67]:
df_target.head()

,artist,title,id,year_chart,date_chart,release_date,collaboration,rank,weeks,isNew,peakPos,lastPos,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms,popularity_artist,popularity_song,genres,album,label,song_lyrics,popularity,target
268122,Imagine Dragons,Radioactive,4G8gkOterJn0Ywt6uhqbhp,2014.0,2014-05-10,2012-09-04,False,49.0,87.0,False,3.0,48.0,0.448,0.784,9,-3.686,1,0.0627,0.1060,0.000108,0.6680,0.236,136.245,4,186813,90,72,"['modern rock', 'pop', 'rock']",Night Visions,Kid Ina Korner / Interscope,Whoah-oh\nWhoah-oh\nWhoah-oh\nWhoah\n\nI'm wak...,NaN,1
267426,AWOLNATION,Sail,5xhQChGGhKLWqBqX4XhtYE,2014.0,2014-03-22,2011,False,45.0,79.0,False,17.0,41.0,0.825,0.435,1,-9.582,1,0.0568,0.4520,0.609000,0.0953,0.243,119.038,4,259102,70,75,"['la indie', 'modern alternative rock', 'moder...",Megalithic Symphony Deluxe,Red Bull Records,Sail!\n\nThis is how I show my love\nI made it...,NaN,1
244363,Jason Mraz,I'm Yours,1EzrEOXmMH3G43AXT1y7pA,2009.0,2009-10-10,2008-05-12,False,48.0,76.0,False,6.0,49.0,0.686,0.457,11,-8.322,1,0.0468,0.5950,0.000000,0.1050,0.718,150.953,4,242186,81,82,"['acoustic pop', 'neo mellow', 'pop', 'pop rock']",We Sing. We Dance. We Steal Things.,Atlantic Records/ATG,"Well, you done done me, and you bet I felt it\...",NaN,1
188602,LeAnn Rimes,How Do I Live,7BD50ATrF3Vab5FQy7vtK8,1998.0,1998-10-10,2003-11-18,False,45.0,69.0,False,2.0,48.0,0.577,0.462,2,-7.989,1,0.0279,0.1280,0.000000,0.0822,0.258,128.303,4,266973,64,68,"['country', 'country dawn', 'dance pop', 'pop ...",Greatest Hits,Curb Records,How do I get through one night without you?\nI...,NaN,1
258774,LMFAO,Party Rock Anthem,0IkKz2J93C94Ei4BvDop7P,2012.0,2012-07-21,2011-01-01,True,49.0,68.0,False,1.0,50.0,0.750,0.727,5,-4.210,0,0.1420,0.0189,0.000000,0.2660,0.359,129.993,4,262173,72,68,"['dance pop', 'edm', 'pop', 'pop rap']",Sorry For Party Rocking,Will I Am / A&M,Party Rock\nYeah\nWoo!\nLet's go!\n\nParty roc...,NaN,1


In [68]:
df_shuffle = df_target.sample(frac=1, random_state=34).reset_index(drop=True)

In [69]:
df_shuffle.head()

,artist,title,id,year_chart,date_chart,release_date,collaboration,rank,weeks,isNew,peakPos,lastPos,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms,popularity_artist,popularity_song,genres,album,label,song_lyrics,popularity,target
0,Future Palace,Illusionist,0ntFhFYgGoRTvQsa6qEaUC,NaN,NaN,2020-03-06,True,NaN,NaN,NaN,NaN,NaN,0.441,0.9200,1,-6.194,1,0.0766,0.003700,0.0000,0.4070,0.578,165.014,4,190909,37,34,['german post-hardcore'],Illusionist,Arising Empire,NaN,NaN,0
1,Ricky Martin,Tal Vez,2VspVkHity8i4H9jammhhR,2003.0,2003-07-19,2003-04-21,False,100.0,8.0,False,74.0,98.0,0.512,0.6150,11,-5.318,1,0.0319,0.216000,0.0000,0.1230,0.193,115.884,4,279226,79,58,"['dance pop', 'latin', 'latin pop', 'mexican p...",Almas Del Silencio,Sony Music Latin,Tal vez sera que esa historia ya tiene final\n...,NaN,1
2,Elastica,Connection,53LZqMzQEnjBkFXPqOq0cD,1995.0,1995-08-12,1995-01-01,False,99.0,20.0,False,53.0,91.0,0.684,0.6940,3,-5.776,1,0.0375,0.000351,0.7000,0.0496,0.882,121.342,4,140480,44,50,"['alternative pop', 'alternative rock', 'britp...",Elastica,DGC,Riding on any wave\nThat is the luck you crave...,NaN,1
3,Blind Lemon Jefferson,Match Box Blues (4446),7hYOjbDYBvUk7RymZ82To3,NaN,NaN,1991,False,NaN,NaN,NaN,NaN,NaN,0.475,0.1570,10,-19.063,1,0.0567,0.976000,0.0348,0.0776,0.519,169.224,4,183066,32,41,"['acoustic blues', 'appalachian folk', 'blues'...",Blind Lemon Jefferson Vol. 2 (1927),Document Records,NaN,NaN,0
4,Elizabeth Barrett Browning,Change Upon Change By Elizabeth Barrett Browning,06Am1qshbnfWRpYilCbBD3,NaN,NaN,2011-07-01,False,NaN,NaN,NaN,NaN,NaN,0.705,0.0944,3,-22.249,1,0.9330,0.831000,0.0000,0.4060,0.433,82.143,4,67069,4,0,['poetry'],Elizabeth Barrett Browning - The Poetry Of,Portable Poetry,NaN,NaN,0


In [70]:
df_shuffle.to_csv("../Data/fusion_songs_1958_2020.csv", index = False, header=True)